In [ ]:
!pip install torch
!pip install pandas
!pip install datasets
!pip install transformers
!pip install scikit-learn
!pip install peft
!pip install codecarbon
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import DatasetDict
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainerCallback
import time
import matplotlib.pyplot as plt
from codecarbon import EmissionsTracker

In [ ]:
# Inspecte les étiquettes du dataset
from datasets import load_dataset

# Charger le dataset
dataset = load_dataset("imdb")

# Vérifie les étiquettes dans le dataset d'entraînement
print(dataset['train'].features['label'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

ClassLabel(names=['neg', 'pos'], id=None)


In [ ]:
# Séparer en train et test
split_datasets = dataset["train"].train_test_split(test_size=0.2)

datasets = DatasetDict({
    "train": split_datasets["train"],
    "test": split_datasets["test"],
})

# Préparer le tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

def tokenize_function(examples):
    tokens = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=256)
    tokens["label"] = examples["label"]
    return tokens

# Appliquer la tokenization
tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Charger le modèle pré-entraîné sans fine-tuning
base_model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base", num_labels=2)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Récupérer les noms des classes directement à partir du dataset
class_names = dataset['train'].features['label'].names

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average=None, zero_division=1)

    # Calcul des métriques globales (moyenne des classes)
    precision_global = precision.mean()
    recall_global = recall.mean()
    f1_macro = f1.mean()  # Ajout du F1-score macro

    metrics = {
        "accuracy": accuracy,
        "precision_global": precision_global,
        "recall_global": recall_global,
        "f1_macro": f1_macro
    }

    # Ajouter les métriques par classe avec les noms explicites
    for i, (p, r, f) in enumerate(zip(precision, recall, f1)):
        metrics[f"precision_{class_names[i]}"] = p
        metrics[f"recall_{class_names[i]}"] = r
        metrics[f"f1_{class_names[i]}"] = f

    return metrics

# Configurer LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Classification de séquence
    inference_mode=False,
    r=8,  # Rang de la décomposition
    lora_alpha=16,  # Facteur d'adaptation
    lora_dropout=0.1,
    target_modules=["query_proj", "key_proj", "value_proj", "dense"]
)

# Appliquer LoRA au modèle
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# Configurer l'entraînement avec fine-tuning LoRA
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    report_to="none",
    overwrite_output_dir=True,
    disable_tqdm=False
)

small_train_dataset = tokenized_datasets["train"]
small_test_dataset = tokenized_datasets["test"]

# Initialisation du Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

tracker = EmissionsTracker()
tracker.start()

train_result = trainer.train()

emissions = tracker.stop()


trainable params: 1,340,930 || all params: 185,764,612 || trainable%: 0.7218


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-ab414c247daf>:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
[codecarbon INFO @ 12:29:13] [setup] RAM Tracking...
[codecarbon INFO @ 12:29:13] [setup] CPU Tracking...
[codecarbon WARNING @ 12:29:13] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecar

Epoch,Training Loss,Validation Loss,Accuracy,Precision Global,Recall Global,F1 Macro,Precision Neg,Recall Neg,F1 Neg,Precision Pos,Recall Pos,F1 Pos
1,0.166800,0.215975,0.935400,0.935398,0.935385,0.935391,0.935196,0.934061,0.934629,0.935599,0.936709,0.936153


[codecarbon INFO @ 12:29:29] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:29:29] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:29:29] Energy consumed for all GPUs : 0.000248 kWh. Total GPU Power : 59.36827903709567 W
[codecarbon INFO @ 12:29:29] 0.000445 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:29:44] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:29:44] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:29:44] Energy consumed for all GPUs : 0.000536 kWh. Total GPU Power : 69.20257198058745 W
[codecarbon INFO @ 12:29:44] 0.000930 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:29:59] Energy consumed for RAM : 0.000059 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:29:59] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codeca

In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from codecarbon import EmissionsTracker

from transformers import AutoModelForSequenceClassification

# Charger le modèle de base
base_model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base", num_labels=2)


freeze_mode = "last_layers"

if freeze_mode == "head":
    for param in base_model.deberta.parameters():
        param.requires_grad = False  # Gèle tout sauf la tête

elif freeze_mode == "last_layers":
    for param in base_model.deberta.parameters():
        param.requires_grad = False  # Gèle tout
    for layer in base_model.deberta.encoder.layer[-2:]:  # Dégeler les 2 dernières couches
        for param in layer.parameters():
            param.requires_grad = True

elif freeze_mode == "embeddings":
    for param in base_model.deberta.embeddings.parameters():
        param.requires_grad = False  # Gèle uniquement les embeddings

elif freeze_mode == "none":
    pass  # Fine-tune complet

# Vérifier les paramètres entraînables
for name, param in base_model.named_parameters():
    print(f"{name}: requires_grad = {param.requires_grad}")

# Fonction de calcul des métriques
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average=None, zero_division=1)

    return {
        "accuracy": accuracy,
        "precision_global": precision.mean(),
        "recall_global": recall.mean(),
        "f1_macro": f1.mean()
    }

# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results_withoutlora",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,  # 3 ou 1 donnent des résultats simillaires
    weight_decay=0.01,
    report_to="none",
    overwrite_output_dir=True,
    disable_tqdm=False
)

# Initialisation du Trainer
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Suivi de l'empreinte carbone
tracker = EmissionsTracker()
tracker.start()

# Entraînement
train_result = trainer.train()

# Fin du suivi carbone
emissions = tracker.stop()


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-114c69a5c62a>:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


deberta.embeddings.word_embeddings.weight: requires_grad = False
deberta.embeddings.LayerNorm.weight: requires_grad = False
deberta.embeddings.LayerNorm.bias: requires_grad = False
deberta.encoder.layer.0.attention.self.query_proj.weight: requires_grad = False
deberta.encoder.layer.0.attention.self.query_proj.bias: requires_grad = False
deberta.encoder.layer.0.attention.self.key_proj.weight: requires_grad = False
deberta.encoder.layer.0.attention.self.key_proj.bias: requires_grad = False
deberta.encoder.layer.0.attention.self.value_proj.weight: requires_grad = False
deberta.encoder.layer.0.attention.self.value_proj.bias: requires_grad = False
deberta.encoder.layer.0.attention.output.dense.weight: requires_grad = False
deberta.encoder.layer.0.attention.output.dense.bias: requires_grad = False
deberta.encoder.layer.0.attention.output.LayerNorm.weight: requires_grad = False
deberta.encoder.layer.0.attention.output.LayerNorm.bias: requires_grad = False
deberta.encoder.layer.0.intermediate.

[codecarbon INFO @ 12:53:59] [setup] RAM Tracking...
[codecarbon INFO @ 12:53:59] [setup] CPU Tracking...
[codecarbon WARNING @ 12:53:59] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 12:54:00] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 12:54:00] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 12:54:00] [setup] GPU Tracking...
[codecarbon INFO @ 12:54:00] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 12:54:00] >>> Tracker's metadata:
[codecarbon INFO @ 12:54:00]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 12:54:00]   Python version: 3.11.11
[codecarbon INFO @ 12:54:00]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 12:54:00]   Available RAM : 12.675 GB
[codecarbon INFO @ 12:54:00]   CPU count: 2
[codeca

Epoch,Training Loss,Validation Loss,Accuracy,Precision Global,Recall Global,F1 Macro
1,0.139700,0.184597,0.941200,0.941274,0.941139,0.941185


[codecarbon INFO @ 12:54:16] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:54:16] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:54:16] Energy consumed for all GPUs : 0.000285 kWh. Total GPU Power : 68.35539409829421 W
[codecarbon INFO @ 12:54:16] 0.000482 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:54:31] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:54:31] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:54:31] Energy consumed for all GPUs : 0.000574 kWh. Total GPU Power : 69.40860322839146 W
[codecarbon INFO @ 12:54:31] 0.000968 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:54:46] Energy consumed for RAM : 0.000059 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:54:46] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codeca

In [ ]:
def plot_loss(trainer):
    """
    Affiche la courbe de la loss en utilisant les steps des logs.
    """
    history = pd.DataFrame(trainer.state.log_history)

    if not history.empty and 'loss' in history:
        plt.figure(figsize=(12, 5))

        # Vérifier s'il y a des logs de loss
        loss_history = history.dropna(subset=['loss'])  # Supprime les lignes sans loss

        plt.plot(loss_history['step'], loss_history['loss'], label="Loss d'entraînement",
                 marker='o', linestyle='-', color='blue', markersize=3)

        plt.xlabel("Steps")
        plt.ylabel("Loss")
        plt.title("Évolution de la Loss")
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.savefig("./results_withoutlora/learning_curves.png")
        plt.show()

In [ ]:
plot_loss(trainer)

In [ ]:
def generate_metrics_report(trainer, final_metrics, training_args, lora_config, emissions):
    """
    Génère un rapport compact avec les métriques clés.
    """
    def safe_format(value):
        return f"{value:.4f}" if isinstance(value, (int, float)) else str(value)

    history = pd.DataFrame(trainer.state.log_history)

    # Calcul du temps moyen par époque
    if 'train_runtime' in history.columns and 'epoch' in history.columns:
        epoch_times = history[history['epoch'].notna()].groupby('epoch')['train_runtime'].mean()
        mean_epoch_time = epoch_times.mean() if not epoch_times.empty else "N/A"
        epochs_table = epoch_times.to_frame().to_markdown()
    else:
        mean_epoch_time = "N/A"
        epochs_table = "Données non disponibles"

    report = f"""
# Rapport d'entraînement et d'évaluation

## **Résultats**
- **Accuracy** : {safe_format(final_metrics.get('eval_accuracy', 'N/A'))}
- **F1-score** : {safe_format(final_metrics.get('eval_f1_macro', 'N/A'))}
- **Precision globale** : {safe_format(final_metrics.get('eval_precision_global', 'N/A'))}
- **Recall global** : {safe_format(final_metrics.get('eval_recall_global', 'N/A'))}
- **Loss finale** : {safe_format(final_metrics.get('eval_loss', 'N/A'))}

## **Hyperparamètres principaux**
- **Epochs** : {training_args.num_train_epochs}
- **Batch (train/eval)** : {training_args.per_device_train_batch_size} / {training_args.per_device_eval_batch_size}
- **Weight Decay** : {training_args.weight_decay}
- **LoRA Config** : r={lora_config.r}, alpha={lora_config.lora_alpha}, dropout={lora_config.lora_dropout}

## **Temps d'entraînement**
- **Temps moyen par époque** : {safe_format(mean_epoch_time)} sec

## **Empreinte carbone**
- **CO₂ estimé** : {safe_format(emissions)} kg

## **Courbes d'apprentissage**
`./results/learning_curves_lora.png`
"""
    with open("./results/training_report_lora.txt", "w") as f:
        f.write(report)
    print(report)


In [ ]:
import pandas as pd

def generate_metrics_report(trainer, final_metrics, training_args, emissions):
    """
    Génère un rapport compact avec les métriques clés (sans LoRA).
    """
    def safe_format(value):
        return f"{value:.4f}" if isinstance(value, (int, float)) else str(value)

    history = pd.DataFrame(trainer.state.log_history)

    # Calcul du temps moyen par époque
    if 'train_runtime' in history.columns and 'epoch' in history.columns:
        epoch_times = history[history['epoch'].notna()].groupby('epoch')['train_runtime'].mean()
        mean_epoch_time = epoch_times.mean() if not epoch_times.empty else "N/A"
        epochs_table = epoch_times.to_frame().to_markdown()
    else:
        mean_epoch_time = "N/A"
        epochs_table = " Données non disponibles"

    report = f"""
# Rapport d'entraînement et d'évaluation

## **Résultats**
- **Accuracy** : {safe_format(final_metrics.get('eval_accuracy', 'N/A'))}
- **F1-score** : {safe_format(final_metrics.get('eval_f1_macro', 'N/A'))}
- **Precision globale** : {safe_format(final_metrics.get('eval_precision_global', 'N/A'))}
- **Recall global** : {safe_format(final_metrics.get('eval_recall_global', 'N/A'))}
- **Loss finale** : {safe_format(final_metrics.get('eval_loss', 'N/A'))}

## **Hyperparamètres principaux**
- **Epochs** : {training_args.num_train_epochs}
- **Batch (train/eval)** : {training_args.per_device_train_batch_size} / {training_args.per_device_eval_batch_size}
- **Weight Decay** : {training_args.weight_decay}

## **Temps d'entraînement**
- **Temps moyen par époque** : {safe_format(mean_epoch_time)} sec

## **Empreinte carbone**
- **CO₂ estimé** : {safe_format(emissions)} kg

## **Courbes d'apprentissage**
`./results_withoutlora/learning_curves.png`
"""
    with open("./results_withoutlora/training_report.txt", "w") as f:
        f.write(report)
    print(report)


In [ ]:
final_metrics = trainer.evaluate()
generate_metrics_report(trainer, final_metrics, training_args, emissions)
print(final_metrics)

In [ ]:
!zip -r mon_dossier.zip ./

In [ ]:
from google.colab import files
files.download("mon_dossier.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>